# Pruebas básicas
Una vez vistas las clases por las que está formada gymnasium y de que trata, se van a realizar pruebas con varios entornos que existen para ver el funcionamiento general de la librería. <br>

En esta primera prueba se va a lanzar un entorno de ejemplo con en el que las acciones que se van a tomar van a ser random, es decir, no se va a seguir ninguna política ni ningún tipo de entrenamiento. Este ejemplo, es únicamente para ver como sería una salida típica.

In [3]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset() #primer reset necesario

for _ in range(1000): #tiempo que está en ejecución el entorno
    action = env.action_space.sample() #accion random
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated: #finalizada la ejecución del anterior
        observation, info = env.reset()

env.close()

Una posible política para este problema sería que el robot intentase estar centrado, ya que la meta siempre aparece en el medio, un ejemplo para tomar es política podría ser:

In [6]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset() #primer reset necesario

for _ in range(1000): #tiempo que está en ejecución el entorno
    if observation[4] > 0:
        observation, reward, terminated, truncated, info = env.step(3)
    else:
        observation, reward, terminated, truncated, info = env.step(1)

    if terminated or truncated: #finalizada la ejecución del anterior
        observation, info = env.reset()

env.close()

In [4]:
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset() #primer reset necesario
print(observation)
env.close()

[ 0.00572338  1.4127272   0.5797061   0.08030551 -0.00662524 -0.13131213
  0.          0.        ]
